In [1]:
import tensorflow as tf
import numpy as np

In [2]:
epochs = 10
batch = 256

In [3]:
def build_cnn():
    inputs = tf.keras.Input(shape=(28,28,1))
    x = tf.keras.layers.Conv2D(8,(3,3),padding='same',activation='relu')(inputs)
    x = tf.keras.layers.Flatten()(x)
    outputs = tf.keras.layers.Dense(10,activation='softmax')(x)
    model = tf.keras.Model(inputs,outputs)
    return model

## Using numpy array as inputs

In [4]:
(X_train_1,y_train_1),(X_test_1,y_test_1) = tf.keras.datasets.mnist.load_data()
X_train_1 = X_train_1.astype(np.float32)/255.
X_train_1 = np.expand_dims(X_train_1,axis=-1)
X_test_1 = X_test_1.astype(np.float32)/255.
X_test_1 = np.expand_dims(X_test_1,axis=-1)
y_train_1 = tf.keras.utils.to_categorical(y_train_1,10)
y_test_1 = tf.keras.utils.to_categorical(y_test_1,10)

model_1 = build_cnn()
model_1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_1.fit(X_train_1,y_train_1,epochs=epochs,batch_size=batch,validation_data=(X_test_1,y_test_1),verbose=2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 - 2s - loss: 0.4546 - acc: 0.8805 - val_loss: 0.2198 - val_acc: 0.9382
Epoch 2/10
60000/60000 - 2s - loss: 0.1915 - acc: 0.9468 - val_loss: 0.1539 - val_acc: 0.9569
Epoch 3/10
60000/60000 - 2s - loss: 0.1375 - acc: 0.9621 - val_loss: 0.1174 - val_acc: 0.9663
Epoch 4/10
60000/60000 - 2s - loss: 0.1105 - acc: 0.9700 - val_loss: 0.1022 - val_acc: 0.9704
Epoch 5/10
60000/60000 - 2s - loss: 0.0929 - acc: 0.9743 - val_loss: 0.0925 - val_acc: 0.9726
Epoch 6/10
60000/60000 - 2s - loss: 0.0807 - acc: 0.9782 - val_loss: 0.0823 - val_acc: 0.9746
Epoch 7/10
60000/60000 - 2s - loss: 0.0699 - acc: 0.9807 - val_loss: 0.0803 - val_acc: 0.9753
Epoch 8/10
60000/60000 - 2s - loss: 0.0636 - acc: 0.9822 - val_loss: 0.0726 - val_acc: 0.9781
Epoch 9/10
60000/60000 - 2s - loss: 0.0567 - acc: 0.9844 - val_loss: 0.0738 - va

In [5]:
model_1.evaluate(X_test_1,y_test_1)

10000/10000 [==============================] - 0s 26us/sample - loss: 0.0665 - acc: 0.9787


[0.06648708410989493, 0.9787]

## Using tf.data.Dataset as inputs

In [12]:
def preprocessing(img,label):
    img = tf.cast(img,tf.float64)
    img = img/255.
    img = tf.expand_dims(img,axis=-1)
    label = tf.one_hot(label,10,dtype=tf.int32)
    return img,label

In [13]:
(X_train_2,y_train_2),(X_test_2,y_test_2) = tf.keras.datasets.mnist.load_data()

train_ds = tf.data.Dataset.from_tensor_slices((X_train_2,y_train_2)).map(preprocessing,num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(256).batch(batch).repeat(epochs).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test_2,y_test_2)).map(preprocessing,num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(batch).repeat(epochs).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [14]:
model_2 = build_cnn()
model_2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_2.fit(train_ds,epochs=epochs,steps_per_epoch=len(X_train_2)//batch,validation_data=test_ds,validation_steps=5,verbose=2)

Epoch 1/10
234/234 - 6s - loss: 0.4722 - acc: 0.8755 - val_loss: 0.2976 - val_acc: 0.9086
Epoch 2/10
234/234 - 2s - loss: 0.2213 - acc: 0.9374 - val_loss: 0.2078 - val_acc: 0.9344
Epoch 3/10
234/234 - 3s - loss: 0.1595 - acc: 0.9554 - val_loss: 0.1617 - val_acc: 0.9547
Epoch 4/10
234/234 - 3s - loss: 0.1249 - acc: 0.9659 - val_loss: 0.1393 - val_acc: 0.9547
Epoch 5/10
234/234 - 3s - loss: 0.1040 - acc: 0.9719 - val_loss: 0.1257 - val_acc: 0.9625
Epoch 6/10
234/234 - 3s - loss: 0.0902 - acc: 0.9757 - val_loss: 0.1172 - val_acc: 0.9664
Epoch 7/10
234/234 - 3s - loss: 0.0792 - acc: 0.9782 - val_loss: 0.1112 - val_acc: 0.9672
Epoch 8/10
234/234 - 3s - loss: 0.0720 - acc: 0.9800 - val_loss: 0.1048 - val_acc: 0.9680
Epoch 9/10
234/234 - 3s - loss: 0.0648 - acc: 0.9819 - val_loss: 0.1033 - val_acc: 0.9695
Epoch 10/10
234/234 - 3s - loss: 0.0594 - acc: 0.9836 - val_loss: 0.1008 - val_acc: 0.9695


In [8]:
model_2.evaluate(test_ds)

400/400 [==============================] - 5s 12ms/step - loss: 0.0731 - acc: 0.9773


[0.07309556936379522, 0.9773]

## tensorflow distributed training with Keras

In [15]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17055773852347534877
]


In [16]:
(X_train_3,y_train_3),(X_test_3,y_test_3) = tf.keras.datasets.mnist.load_data()

mirrored = tf.distribute.MirroredStrategy()
print('\nnumber of replicas in sync: {}'.format(mirrored.num_replicas_in_sync))

batch_per_replica = batch
global_batch = batch_per_replica * mirrored.num_replicas_in_sync
print('global batch: {}'.format(global_batch))


number of replicas in sync: 1
global batch: 256


In [17]:
train_ds_dist = tf.data.Dataset.from_tensor_slices((X_train_3,y_train_3)).map(preprocessing,num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(256).batch(global_batch).repeat(epochs).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_ds_dist = tf.data.Dataset.from_tensor_slices((X_test_3,y_test_3)).map(preprocessing,num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(global_batch).repeat(epochs).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

with mirrored.scope():
    model_3 = build_cnn()
    model_3.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_3.fit(train_ds_dist,epochs=epochs,steps_per_epoch=len(X_train_3)//batch,validation_data=test_ds_dist,validation_steps=5,verbose=2)

INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CP

In [12]:
model_3.evaluate(test_ds_dist)

400/400 [==============================] - 5s 13ms/step - loss: 0.0772 - acc: 0.9756


[0.07716921950923279, 0.9756]